<a href="https://colab.research.google.com/github/MardelaineA/cpe019_final-project_Acay/blob/main/cpe019_finalproject_acay.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Final Exam: Model Deployment in the Cloud
Name: Acay, Mardelaine Jahleel D. <br>
Course and Section: CPE019S3 <br>
Date Submitted: July 11, 2025 <br>
Instructor:Engr. Robin Valenzuela <br>
Date Performed: July 11, 2025

In [1]:
!pip install tensorflow streamlit opencv-python pillow

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 104.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 83.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 5.7 MB/s eta 0:00:00


In [2]:
!pip install kaggle
!mkdir -p ~/.kaggle
!cp /path/to/your/kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!kaggle datasets download -d pratik2901/multiclass-weather-dataset
!unzip multiclass-weather-dataset.zip

cp: cannot stat '/path/to/your/kaggle.json': No such file or directory
chmod: cannot access '/root/.kaggle/kaggle.json': No such file or directory
Traceback (most recent call last):
  File "/usr/local/bin/kaggle", line 10, in <module>
    sys.exit(main())
             ^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/kaggle/cli.py", line 68, in main
    out = args.func(**command_args)
          ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/kaggle/api/kaggle_api_extended.py", line 1741, in dataset_download_cli
    with self.build_kaggle_client() as kaggle:
         ^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/kaggle/api/kaggle_api_extended.py", line 688, in build_kaggle_client
    username=self.config_values['username'],
             ~~~~~~~~~~~~~~~~~~^^^^^^^^^^^^
KeyError: 'username'
unzip:  cannot find or open multiclass-weather-dataset.zip, multiclass-weather-dataset.zip.zip or multiclass-weather-dataset.zip.ZIP.


In [3]:
import os, shutil
from sklearn.model_selection import train_test_split

multiclass_dataset_dir = '/content/drive/MyDrive/CPE 019_FINALS EXAM/archive(1)/Multi-class Weather Dataset' # Corrected directory name
original_dir = '/content/drive/MyDrive/CPE 019_FINALS EXAM/archive(1)'
os.makedirs(original_dir, exist_ok=True)

classes = ['Sunrise', 'Shine', 'Rain', 'Cloudy']

for c in classes:
    os.makedirs(os.path.join(original_dir, 'train', c), exist_ok=True)
    os.makedirs(os.path.join(original_dir, 'test', c), exist_ok=True)

for c in classes:
    src_dir = os.path.join(multiclass_dataset_dir, c)
    all_images = os.listdir(src_dir)
    train_files, test_files = train_test_split(all_images, test_size=0.2, random_state=42)

    for f in train_files:
        shutil.copy(os.path.join(src_dir, f), os.path.join(original_dir, 'train', c, f))
    for f in test_files:
        shutil.copy(os.path.join(src_dir, f), os.path.join(original_dir, 'test', c, f))

In [4]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

train_path = '/content/drive/MyDrive/CPE 019_FINALS EXAM/archive(1)/train'
test_path = '/content/drive/MyDrive/CPE 019_FINALS EXAM/archive(1)/test'

train_gen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    zoom_range=0.3,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

test_gen = ImageDataGenerator(rescale=1./255)

train_generator = train_gen.flow_from_directory(
    train_path, target_size=(75, 75), batch_size=32, class_mode='categorical')

test_generator = test_gen.flow_from_directory(
    test_path, target_size=(75, 75), batch_size=32, class_mode='categorical')

Found 899 images belonging to 4 classes.
Found 226 images belonging to 4 classes.


In [7]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=(75,75,3)),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dropout(0.4),
    tf.keras.layers.Dense(4, activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

early_stop = EarlyStopping(monitor='val_accuracy', patience=10, restore_best_weights=True)
checkpoint = ModelCheckpoint("best_model.h5", monitor='val_accuracy', save_best_only=True)

history = model.fit(train_generator, validation_data=test_generator, epochs=50, callbacks=[early_stop, checkpoint])

model.save("Weather multi-class_model.h5")

loss, acc = model.evaluate(test_generator)
print(f"Final Accuracy: {acc * 100:.2f}%")

Epoch 1/50
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 541ms/step - accuracy: 0.4342 - loss: 1.2403

29/29 ━━━━━━━━━━━━━━━━━━━━ 22s 647ms/step - accuracy: 0.4368 - loss: 1.2343 - val_accuracy: 0.5752 - val_loss: 0.7406
Epoch 2/50
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 533ms/step - accuracy: 0.6937 - loss: 0.7370

29/29 ━━━━━━━━━━━━━━━━━━━━ 19s 605ms/step - accuracy: 0.6940 - loss: 0.7367 - val_accuracy: 0.8496 - val_loss: 0.5422
Epoch 3/50
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 537ms/step - accuracy: 0.7762 - loss: 0.5651

29/29 ━━━━━━━━━━━━━━━━━━━━ 18s 612ms/step - accuracy: 0.7758 - loss: 0.5658 - val_accuracy: 0.8628 - val_loss: 0.4434
Epoch 4/50
29/29 ━━━━━━━━━━━━━━━━━━━━ 18s 629ms/step - accuracy: 0.7921 - loss: 0.5735 - val_accuracy: 0.8363 - val_loss: 0.3977
Epoch 5/50
29/29 ━━━━━━━━━━━━━━━━━━━━ 19s 622ms/step - accuracy: 0.7838 - loss: 0.5383 - val_accuracy: 0.8451 - val_loss: 0.4317
Epoch 6/50
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 522ms/step - accuracy: 0.7848 - loss: 0.5689

29/29 ━━━━━━━━━━━━━━━━━━━━ 19s 594ms/step - accuracy: 0.7849 - loss: 0.5688 - val_accuracy: 0.9071 - val_loss: 0.3302
Epoch 7/50
29/29 ━━━━━━━━━━━━━━━━━━━━ 17s 592ms/step - accuracy: 0.8308 - loss: 0.4682 - val_accuracy: 0.8850 - val_loss: 0.3132
Epoch 8/50
29/29 ━━━━━━━━━━━━━━━━━━━━ 22s 631ms/step - accuracy: 0.8711 - loss: 0.3910 - val_accuracy: 0.9071 - val_loss: 0.2780
Epoch 9/50
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 531ms/step - accuracy: 0.8620 - loss: 0.4092

29/29 ━━━━━━━━━━━━━━━━━━━━ 18s 609ms/step - accuracy: 0.8618 - loss: 0.4089 - val_accuracy: 0.9292 - val_loss: 0.2531
Epoch 10/50
29/29 ━━━━━━━━━━━━━━━━━━━━ 18s 620ms/step - accuracy: 0.8867 - loss: 0.3463 - val_accuracy: 0.9204 - val_loss: 0.2405
Epoch 11/50
29/29 ━━━━━━━━━━━━━━━━━━━━ 23s 725ms/step - accuracy: 0.8612 - loss: 0.3629 - val_accuracy: 0.9248 - val_loss: 0.2330
Epoch 12/50
29/29 ━━━━━━━━━━━━━━━━━━━━ 17s 590ms/step - accuracy: 0.8771 - loss: 0.3239 - val_accuracy: 0.9115 - val_loss: 0.2707
Epoch 13/50
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 517ms/step - accuracy: 0.8516 - loss: 0.3992

29/29 ━━━━━━━━━━━━━━━━━━━━ 17s 590ms/step - accuracy: 0.8517 - loss: 0.3989 - val_accuracy: 0.9381 - val_loss: 0.2003
Epoch 14/50
29/29 ━━━━━━━━━━━━━━━━━━━━ 19s 662ms/step - accuracy: 0.8844 - loss: 0.3068 - val_accuracy: 0.9071 - val_loss: 0.2212
Epoch 15/50
29/29 ━━━━━━━━━━━━━━━━━━━━ 18s 604ms/step - accuracy: 0.8749 - loss: 0.3118 - val_accuracy: 0.9204 - val_loss: 0.2384
Epoch 16/50
29/29 ━━━━━━━━━━━━━━━━━━━━ 17s 600ms/step - accuracy: 0.8850 - loss: 0.3253 - val_accuracy: 0.9115 - val_loss: 0.2201
Epoch 17/50
29/29 ━━━━━━━━━━━━━━━━━━━━ 18s 604ms/step - accuracy: 0.8596 - loss: 0.3709 - val_accuracy: 0.9204 - val_loss: 0.2637
Epoch 18/50
29/29 ━━━━━━━━━━━━━━━━━━━━ 24s 715ms/step - accuracy: 0.8826 - loss: 0.3392 - val_accuracy: 0.9248 - val_loss: 0.2095
Epoch 19/50
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 533ms/step - accuracy: 0.8809 - loss: 0.2893

29/29 ━━━━━━━━━━━━━━━━━━━━ 18s 605ms/step - accuracy: 0.8809 - loss: 0.2899 - val_accuracy: 0.9469 - val_loss: 0.1638
Epoch 20/50
29/29 ━━━━━━━━━━━━━━━━━━━━ 17s 593ms/step - accuracy: 0.9259 - loss: 0.2268 - val_accuracy: 0.9248 - val_loss: 0.1717
Epoch 21/50
29/29 ━━━━━━━━━━━━━━━━━━━━ 18s 601ms/step - accuracy: 0.8635 - loss: 0.3493 - val_accuracy: 0.9204 - val_loss: 0.2298
Epoch 22/50
29/29 ━━━━━━━━━━━━━━━━━━━━ 18s 617ms/step - accuracy: 0.8547 - loss: 0.3792 - val_accuracy: 0.9204 - val_loss: 0.1899
Epoch 23/50
29/29 ━━━━━━━━━━━━━━━━━━━━ 18s 611ms/step - accuracy: 0.8850 - loss: 0.2865 - val_accuracy: 0.9115 - val_loss: 0.2424
Epoch 24/50
29/29 ━━━━━━━━━━━━━━━━━━━━ 18s 620ms/step - accuracy: 0.8653 - loss: 0.3473 - val_accuracy: 0.9336 - val_loss: 0.1733
Epoch 25/50
29/29 ━━━━━━━━━━━━━━━━━━━━ 17s 599ms/step - accuracy: 0.9070 - loss: 0.2581 - val_accuracy: 0.9292 - val_loss: 0.1703
Epoch 26/50
29/29 ━━━━━━━━━━━━━━━━━━━━ 21s 609ms/step - accuracy: 0.9099 - loss: 0.2393 - val_accuracy

8/8 ━━━━━━━━━━━━━━━━━━━━ 2s 225ms/step - accuracy: 0.9265 - loss: 0.1970
Final Accuracy: 94.69%
